In [2]:
#coding:utf-8
__author__ = 'Mr.数据杨'
__explain__ = '爬虫数据存储MySQL模板'

import requests
from bs4 import BeautifulSoup
import pymysql

In [8]:
mysql_engine = {
    "host":"localhost", 
    "database":"sampledb", 
    "user":"root", 
    "password":"admin",
    "charset":"utf8"}

#创建数据库
def creat_database_sampledb():
    config_root = {
        "host": "localhost",
        "user": "root",
        "password": "admin"}
    sql = "Create Database If Not Exists sampledb CHARSET=utf8"
    conn = pymysql.connect(**config_root)  # 打开数据库连接
    try:
        with conn.cursor() as cursor:  # 使用cursor()方法获取操作游标,并在语句结束自动关闭
            cursor.execute(sql)  # 执行SQL
            conn.commit()  # 提交
    finally:
        conn.close()
creat_database_sampledb()

def createNewsTable():
    createTbSql = (
        "Create Table If Not Exists NewsTable( "
        "id int primary key auto_increment, "
        "title varchar(100), "
        "url varchar(100), "
        "date date,"
        "text text)")
    try:
        corsor = pymysql.connect(**mysql_engine)
        with corsor.cursor() as cursor:
            cursor.execute(createTbSql)
            corsor.commit()
    finally:
        corsor.close()
createNewsTable()

C:\Users\AERO\Anaconda3\lib\site-packages\pymysql\cursors.py:166: Warning: (1007, "Can't create database 'sampledb'; database exists")
  result = self._query(query)


In [13]:
url = "http://www.cwestc.com/MroeNews.aspx?gd=1"
html = requests.get(url)
soup = BeautifulSoup(html.text,'lxml')
lptable = soup.find('table',width='780')
for i in lptable.find_all("td",width="680"): 
    OnePage = []
    Title = i.b.strong.a.string  # title字段
    Url =  "http://www.cwestc.com"+i.find('a')['href'] # url字段
    Date =  Url.split("/")[4]  # 日期字段
    # 根据url抓取正文
    content = requests.get(Url)
    content.encoding = "utf-8"
    soup = BeautifulSoup(content.text,'html.parser')
    try:
        NewsBody = str(soup.find('td',{'class':'newsContent'}))
    except:
        NewsBody = "无内容"
    OnePage.append((Title,Url,Date,NewsBody))
    # 写入MySQL
    sql = "insert into NewsTable(id,title,url,date,text) values(Null,%s,%s,%s,%s)"
    conn = pymysql.connect(**mysql_engine)
    try:
        with conn.cursor() as cursor:
            cursor.executemany(sql, OnePage)
            conn.commit()
    finally:
        conn.close()  